In [24]:
# dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from datetime import datetime
from config import weather_api_key
from citipy import citipy

In [25]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [26]:
# unzip lat_lngs
coordinates = list(lat_lngs)

In [31]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

606

In [28]:
# building API request URL
url = "https://api.openweathermap.org/data/2.5/weather?" + "&appid=" + weather_api_key

In [36]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | calama
Processing Record 2 of Set 1 | atuona
Processing Record 3 of Set 1 | sitka
Processing Record 4 of Set 1 | narsaq
Processing Record 5 of Set 1 | rocha
Processing Record 6 of Set 1 | hobart
Processing Record 7 of Set 1 | mocoa
Processing Record 8 of Set 1 | klaksvik
Processing Record 9 of Set 1 | taksimo
Processing Record 10 of Set 1 | port elizabeth
Processing Record 11 of Set 1 | vila do maio
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | port alfred
Processing Record 14 of Set 1 | puerto ayora
Processing Record 15 of Set 1 | jamestown
Processing Record 16 of Set 1 | marshfield
Processing Record 17 of Set 1 | nantucket
Processing Record 18 of Set 1 | provideniya
Processing Record 19 of Set 1 | hualmay
Processing Record 20 of Set 1 | bestobe
Processing Record 21 of Set 1 | marcona
City not found. Skipping...
Processing Record 22 of Set 1 | hilo
Processing Record 23 

In [38]:
# confirm dataset size is >=500
len(city_data)

560

In [39]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Calama,-22.4667,-68.9333,294.42,3,0,9.77,CL,2022-09-26 17:41:15
1,Atuona,-9.8000,-139.0333,298.02,73,29,10.03,PF,2022-09-26 17:38:36
2,Sitka,57.0531,-135.3300,290.09,82,100,6.17,US,2022-09-26 17:41:16
3,Narsaq,60.9167,-46.0500,283.27,93,100,3.50,GL,2022-09-26 17:40:58
4,Rocha,-34.4833,-54.3333,291.24,81,93,3.19,UY,2022-09-26 17:41:17
5,Hobart,-42.8794,147.3294,283.74,90,100,0.51,AU,2022-09-26 17:41:18
6,Mocoa,1.1493,-76.6466,302.17,61,84,1.67,CO,2022-09-26 17:41:19
7,Klaksvik,62.2266,-6.5890,277.66,96,100,20.28,FO,2022-09-26 17:41:21
8,Taksimo,56.3444,114.8794,273.16,73,64,1.01,RU,2022-09-26 17:41:21
9,Port Elizabeth,-33.9180,25.5701,292.09,91,100,6.17,ZA,2022-09-26 17:41:23


In [40]:
# reorder city_data_df columns
reordered_col = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[reordered_col]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Calama,CL,2022-09-26 17:41:15,-22.4667,-68.9333,294.42,3,0,9.77
1,Atuona,PF,2022-09-26 17:38:36,-9.8000,-139.0333,298.02,73,29,10.03
2,Sitka,US,2022-09-26 17:41:16,57.0531,-135.3300,290.09,82,100,6.17
3,Narsaq,GL,2022-09-26 17:40:58,60.9167,-46.0500,283.27,93,100,3.50
4,Rocha,UY,2022-09-26 17:41:17,-34.4833,-54.3333,291.24,81,93,3.19


In [41]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")